In [ ]:
from google.colab import drive
import tensorflow as tf
import os
import numpy as np
drive.mount('/content/drive')
PATH_DATA = '/content/drive/MyDrive/WMA_faces/'
directory_train = os.path.join(PATH_DATA,'train')
directory_val = os.path.join(PATH_DATA,'val')
train = tf.keras.utils.image_dataset_from_directory(directory_train,labels='inferred')
val = tf.keras.utils.image_dataset_from_directory(directory_val,labels='inferred')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Found 93 files belonging to 4 classes.
Found 25 files belonging to 4 classes.


In [ ]:
normalization_layer = tf.keras.layers.Rescaling(1./255)
normalized_ds = train.map(lambda x, y: (normalization_layer(x),y))
image_batch, labels_batch = next(iter(normalized_ds))
first_image = image_batch[0]
# Notice the pixel values are now in `[0,1]`.
print(np.min(first_image), np.max(first_image))


0.004363655 0.9985434


In [ ]:
num_classes = 4

model = tf.keras.Sequential([
  tf.keras.layers.Rescaling(1./255),
  tf.keras.layers.Conv2D(32, 3, activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Conv2D(32, 3, activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Conv2D(32, 3, activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(num_classes)
])

model.compile(
  optimizer='adam',
  loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
  metrics=['accuracy'])

model.fit(
  train,
  validation_data=val,
  epochs=3
)


Epoch 1/3
3/3 [==============================] - 14s 4s/step - loss: 2.2159 - accuracy: 0.2043 - val_loss: 1.4784 - val_accuracy: 0.2800
Epoch 2/3
3/3 [==============================] - 12s 4s/step - loss: 1.3896 - accuracy: 0.3011 - val_loss: 1.3792 - val_accuracy: 0.2400
Epoch 3/3
3/3 [==============================] - 10s 3s/step - loss: 1.3362 - accuracy: 0.4086 - val_loss: 1.3566 - val_accuracy: 0.3600


In [1]:
import cv2

cap = cv2.VideoCapture(1)

ret, frame = cap.read()


img_array = tf.keras.utils.img_to_array(frame)
img_array = tf.expand_dims(img_array, 0) # Create a batch

predictions = model.predict(img_array)
score = tf.nn.softmax(predictions[0])

print(
    "This image most likely belongs to {} with a {:.2f} percent confidence."
    .format(class_names[np.argmax(score)], 100 * np.max(score))
)


NameError: ignored